Excercise

## PV, CHP, Bio, Natural-gas, District heating

### 1)Import Packages

In [1]:
import pypsa
import pandapower as pp
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
import warnings
import subprocess
from shapely.errors import ShapelyDeprecationWarning
import logging

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
logging.getLogger("pypsa.pf").setLevel(logging.WARNING)
plt.rc("figure", figsize=(10, 8))

c:\Users\moino\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\moino\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\moino\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


### 2)Create a network and set Snapshots

In [ ]:
#availability = pd.read_csv("availability_df.csv", index_col=0)
#availability.head(2)

In [2]:
# Create a new PyPSA network
network = pypsa.Network()
#snapshots1=pd.date_range(start="2023-01-01 00:00", end="2023-01-31 23:00", freq="h")
#network.set_snapshots(pd.date_range("2024-01-01", periods=8760, freq_"h"))
network.set_snapshots=pd.read_csv('snapshots.csv')
#network.set_snapshots(snapshots1)
solver='highs'

### 3)Add Buses

In [ ]:
Bus= pd.read_csv('buses.csv',header=0)
network.madd(
    "Bus",
    names= Bus.Bus,
    v_nom=list(Bus.v_nom),
    carrier=list(Bus.carrier),
    x=list(Bus.x),
    y=list(Bus.y),
)
network.buses.head()

### 4)Add generators

In [ ]:
generators=pd.read_csv('powerplants.csv',header=0)
generators.head()

In [ ]:
# Read excel file which contains Non renewable generators data
generators=pd.read_csv('powerplants.csv',header=0)
gen_p_max_pu=pd.read_csv('availability_df.csv',parse_dates=True)
#gen_p_max_pu['datetime']=pd.to_datetime(gen_p_max_pu['datetime'],format="%d-Xm-Xy %H:%M")
#gen_p_max_pu.set_index('datetime',inplace=True)
network.madd(
    "Generator",
    names= generators.name,
    bus=list(generators.bus),
    #carrier=list(powerplants.carrier),
    #p_nom=list(generators.p_nom),
    p_nom_extendable=list(generators.p_nom_extendable),
    #p_max_pu=gen_p_max_pu,
    capital_cost=list(generators.capital_cost),
    marginal_cost=list(generators.marginal_cost),
    efficiency=list(generators.efficiency)
) 
network.generators.head()

In [ ]:
network.generators_t.p_max_pu=gen_p_max_pu

In [ ]:
network.generators_t.p_max_pu.head(2)

### 5)Add load

In [ ]:
# Read excel file which contains Non renewable generators data
load=pd.read_csv('demand_units.csv',header=0)
load_p_set = pd.read_csv('demand_df.csv', parse_dates=True)
load_p_set.set_index('datetime', inplace=True)
#demand_df=pd
network.madd(
    "Load",
    names= load.load,
    bus=list(load.bus),
    #p_set=list(load.p_set),
)
network.loads.p_set.head(2)

In [ ]:
# Read excel file which contains Non renewable generators data
load=pd.read_csv('demand_units.csv',header=0)
load_p_set = pd.read_csv('demand_df.csv', parse_dates=True)
load_p_set.set_index('datetime', inplace=True)
#demand_df=pd
network.madd(
    "Load",
    names= load.load,
    bus=list(load.bus),
    #p_set=list(load.p_set),
)
network.loads.head()

In [ ]:
lines=pd.read_csv('lines.csv',header=0)
network.madd(
    "Line",
    names= lines.Line,
    bus0=list(lines.bus0),
    bus1=list(lines.bus1),
    s_nom=list(lines.s_nom),
    s_nom_extendable=list(lines.s_nom),
    r=list(lines.r),
    x=list(lines.x),
)
network.lines.head()

### 6)Add links

In [ ]:
links = pd.read_csv('links.csv', header=0)
network.madd(
    "Link",
    names=links.Link,
    bus0=list(links.bus0),
    bus1=list(links.bus1),
    #bus2=list(links.bus2),
    p_nom=list(links.p_nom),
    p_nom_extendable=list(links.p_nom_extendable),
    efficiency=list(links.efficiency),
    #efficiency2=list(links.efficiency2),
    capital_cost=list(links.capital_cost),
)
network.links

### 7)Add carriers

In [ ]:
carriers = pd.read_csv('carrier.csv', header=0)
network.madd(
    "Carrier",
    names=carriers.carrier,
    co2_emissions=list(carriers.co2_emissions),
    nice_name=list(carriers.carrier),
)

### 8)Storages

In [ ]:
stores = pd.read_csv('stores.csv', header=0)
network.madd(
    "Store",
    names=stores.name,
    e_initial=list(stores.e_initial),
    e_nom=list(stores.e_nom),
    marginal_cost=list(stores.marginal_cost),
    bus=list(stores.bus),
    e_cyclic=list(stores.e_cyclic),
    e_nom_extendable=list(stores.e_nom_extendable),
)
network.stores

storage_units = pd.read_csv('storage_units.csv', header=0)
network.madd(
    "Storage_units",
    names=stores.name,
    e_initial=list(stores.e_initial),
    e_nom=list(stores.e_nom),
    marginal_cost=list(stores.marginal_cost),
    bus=list(stores.bus),
    e_cyclic=list(stores.e_cyclic),
    e_nom_extendable=list(stores.e_nom_extendable),
)
network.stores

### 9)Global constraints

In [ ]:
network.add("GlobalConstraint", "co2_limit", sense="<=", constant=1000000.0)

In [ ]:
network.optimize(solver_name='highs')

In [ ]:
network.pf()

In [ ]:
# Your DataFrame df
df = pd.concat([network.generators_t.p.loc[0],
        network.links_t.p0.loc[0],
        network.loads_t.p.loc[0],
    ],
    keys=["Generators", "Links", "Line"],
    names=["Component", "index"],
).reset_index(name="Production")

# Plotting using PyPSA
fig, ax = plt.subplots(figsize=(13, 7))

# Plot generators
df_generators = df[df["Component"] == "Generators"]
ax.bar(df_generators["index"], df_generators["Production"], label="Generators")

# Plot links
df_links = df[df["Component"] == "Links"]
ax.bar(df_links["index"], df_links["Production"], label="Links")

# Plot loads
df_loads = df[df["Component"] == "Line"]
ax.bar(df_loads["index"], df_loads["Production"], label="Line")

# Customize the plot
ax.set_xlabel("Component")
ax.set_ylabel("Production (MW)")
ax.set_title("Power Production by Component")
ax.legend()

# Show the plot
plt.show()
